<h4><b>Make DataLoader</b>

In [4]:
import torch
import torchvision
from torchvision import datasets, transforms

train_dir = '.\\dataset\\training_dataset'
test_dir = '.\\dataset\\validation_dataset'

train_transforms = transforms.Compose(  [   transforms.Resize((224,224)),
                                            transforms.ToTensor(),                                
                                            torchvision.transforms.Normalize(
                                                mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225],
                                            ),
                                            transforms.RandomErasing()
                                        ]   )

test_transforms = transforms.Compose(   [   transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            torchvision.transforms.Normalize(
                                            mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225],
                                            ),
                                            transforms.RandomErasing()
                                        ]   )

train_data = datasets.ImageFolder(train_dir,transform=train_transforms)
test_data = datasets.ImageFolder(test_dir,transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=32)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=32)


<h4><b>Making a Training Process</b>

In [5]:
def training_process(model, optimizer, loss_func):
    
    def train_step(x,y):
        #make prediction
        yhat = model(x)
        #enter train mode
        model.train()
        #compute loss
        loss = loss_func(yhat,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #optimizer.cleargrads()

        return loss
    
    return train_step

<h4><b>Build the ResNet50 Model</b>

In [6]:
from torchvision import datasets, models, transforms
import torch.nn as nn
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50_model = models.resnet50(pretrained=True)

#freeze all params
for params in resnet50_model.parameters():
    params.requires_grad_ = False

#add layers in model
num_ftrs = resnet50_model.fc.in_features
resnet50_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),  # 修改輸出維度為 1，這裡是為了二分類
    nn.Sigmoid()  # 加上 Sigmoid 函數
)
resnet50_model = resnet50_model.to(device)
summary(resnet50_model, input_size=(3, 224, 224), device = 'cuda')


<h4><b>Loss, Optimizer, and Train Step</b>

In [ ]:
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

#loss
loss_fn = BCEWithLogitsLoss() #binary cross entropy with sigmoid, so no need to use sigmoid in the model

#optimizer
optimizer = torch.optim.Adam(resnet50_model.fc.parameters()) 

#train step
train_step = training_process(resnet50_model, optimizer, loss_fn)

<h4><b>Train the Model</b>

How to Load model to predict an image is dog or cat?<br>
<href>https://blog.csdn.net/qq_41167777/article/details/109013155</href>

In [ ]:
from tqdm import tqdm


losses = []
val_losses = []

epoch_train_losses = []
epoch_test_losses = []

n_epochs = 40
early_stopping_tolerance = 3
early_stopping_threshold = 0.03

for epoch in range(n_epochs):
  epoch_loss = 0
  for i ,data in tqdm(enumerate(trainloader), total = len(trainloader)): #iterate ove batches
    x_batch , y_batch = data
    x_batch = x_batch.to(device) #move to gpu
    y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
    y_batch = y_batch.to(device) #move to gpu


    loss = train_step(x_batch, y_batch)
    epoch_loss += loss/len(trainloader)
    losses.append(loss)
    
  epoch_train_losses.append(epoch_loss)
  print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))

  #validation doesnt requires gradient
  with torch.no_grad():
    cum_loss = 0
    for x_batch, y_batch in testloader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
      y_batch = y_batch.to(device)

      #model to eval mode
      resnet50_model.eval()

      yhat = resnet50_model(x_batch)
      val_loss = loss_fn(yhat,y_batch)
      cum_loss += loss/len(testloader)
      val_losses.append(val_loss.item())


    epoch_test_losses.append(cum_loss)
    print('Epoch : {}, val loss : {}'.format(epoch+1,cum_loss))  
    
    best_loss = min(epoch_test_losses)
    
    #save best model
    if cum_loss <= best_loss:
      best_model_wts = resnet50_model.state_dict()
    
    #early stopping
    early_stopping_counter = 0
    if cum_loss > best_loss:
      early_stopping_counter +=1

    if (early_stopping_counter == early_stopping_tolerance) or (best_loss <= early_stopping_threshold):
      print("/nTerminating: early stopping")
      break #terminate training
    
#load best model
resnet50_model.load_state_dict(best_model_wts)
torch.save(resnet50_model, './RE_model.pt')
torch.save(best_model_wts, './RE_best_model_wts.pt')